<a href="https://colab.research.google.com/github/rednaviblue/LearningsPublic/blob/Geomatics/Celestial_Coordinate_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TO DO LIST TO EXECUTE THE PROGRAM**
1) Run the first cell of code
2) Run the second cell of code

To Run the code just click the horizontal triangle THE GODDAMN **Play Button**: THIS "▶"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [62]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.graph_objs import FigureWidget
import ipywidgets as widgets
from IPython.display import display

R = 10.0

def sph_to_cart(alt_deg, az_deg, radius=R):
    h = np.radians(alt_deg)
    A = np.radians(az_deg)
    x = radius * np.cos(h) * np.sin(A)
    y = radius * np.cos(h) * np.cos(A)
    z = radius * np.sin(h)
    return x, y, z

def circle_xy(radius=R, z_level=0, n=200):
    theta = np.linspace(0, 2*np.pi, n)
    return radius*np.cos(theta), radius*np.sin(theta), np.full_like(theta, z_level)

def almucantar_trace(alt_deg, radius=R):
    h = np.radians(alt_deg)
    theta = np.linspace(0, 2*np.pi, 240)
    r = radius * np.cos(h)
    z = np.full_like(theta, radius * np.sin(h))
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return go.Scatter3d(x=x, y=y, z=z, mode="lines",
                        line=dict(color="gray", width=2, dash="dot"),
                        name=f"Almucantar {alt_deg}°", visible=False)

def vertical_circle_trace(az_deg, radius=R):
    phi = np.radians(az_deg)
    theta = np.linspace(0, np.pi, 200)
    x = radius * np.sin(theta) * np.sin(phi)
    y = radius * np.sin(theta) * np.cos(phi)
    z = radius * np.cos(theta)
    return go.Scatter3d(x=x, y=y, z=z, mode="lines",
                        line=dict(color="blue", width=3),
                        name=f"Vertical Circle {az_deg}°", visible=False)

u, v = np.mgrid[0:2*np.pi:60j, 0:np.pi:30j]
xs = R * np.cos(u) * np.sin(v)
ys = R * np.sin(u) * np.sin(v)
zs = R * np.cos(v)
sphere = go.Surface(x=xs, y=ys, z=zs, opacity=0.12, showscale=False, name="Celestial Sphere", hoverinfo="skip")

xh, yh, zh = circle_xy(R, 0)
horizon = go.Scatter3d(x=xh, y=yh, z=zh, mode="lines", line=dict(color="saddlebrown", width=3), name="Horizon")

zenith = go.Scatter3d(x=[0], y=[0], z=[R], mode="markers+text",
                      marker=dict(size=6, color="green"), text=["Zenith"], textposition="top center", showlegend=False)
nadir = go.Scatter3d(x=[0], y=[0], z=[-R], mode="markers+text",
                     marker=dict(size=6, color="purple"), text=["Nadir"], textposition="bottom center", showlegend=False)

north = go.Scatter3d(x=[0], y=[R], z=[0], mode="text", text=["N"], showlegend=False)
south = go.Scatter3d(x=[0], y=[-R], z=[0], mode="text", text=["S"], showlegend=False)
east = go.Scatter3d(x=[R], y=[0], z=[0], mode="text", text=["E"], showlegend=False)
west = go.Scatter3d(x=[-R], y=[0], z=[0], mode="text", text=["W"], showlegend=False)

alm_traces = [almucantar_trace(60), almucantar_trace(30), almucantar_trace(-30), almucantar_trace(-60)]

az_options = list(range(0, 360, 45))
vert_traces = [vertical_circle_trace(az) for az in az_options]

default_az = 0
default_alt = 45
x_star, y_star, z_star = sph_to_cart(default_alt, default_az)
star_trace = go.Scatter3d(x=[x_star], y=[y_star], z=[z_star],
                          mode="markers+text", marker=dict(size=6, color="red"),
                          text=["Star"], textposition="top center", name="Star")

fig = FigureWidget(data=[sphere, horizon, zenith, nadir, north, south, east, west] + alm_traces + vert_traces + [star_trace])

idx = {
    "alm_start": 8,
    "vert_start": 12,
    "star": 20
}

fig.data[idx["vert_start"] + az_options.index(default_az)].visible = True

fig.update_layout(
    scene=dict(xaxis=dict(showbackground=False, visible=False),
               yaxis=dict(showbackground=False, visible=False),
               zaxis=dict(showbackground=False, visible=False),
               aspectmode='data'),
    margin=dict(l=0, r=0, b=0, t=30),
    legend=dict(x=0.75, y=0.85, bgcolor="rgba(255,255,255,0.5)"),
    title="Horizon System — vertical circle & star placement"
)

vert_dropdown = widgets.Dropdown(options=[(f"{a}°", a) for a in az_options],
                                 value=default_az, description="Vertical Azimuth:")
alm_checkbox = widgets.Checkbox(value=False, description="Show Almucantars (±30°, ±60°)")
alt_slider = widgets.FloatSlider(value=default_alt, min=-89, max=89, step=0.5, description="Star Alt (°):")
result_box = widgets.HTML(value="")

def update_star_position(alt_deg, az_deg):
    x, y, z = sph_to_cart(alt_deg, az_deg)
    fig.data[idx["star"]].x = [x]
    fig.data[idx["star"]].y = [y]
    fig.data[idx["star"]].z = [z]
    zdist = 90 - alt_deg
    result_box.value = (f"<b>Star (constrained to vertical circle)</b><br>"
                        f"Altitude (h): {alt_deg:.2f}°<br>"
                        f"Azimuth (A): {az_deg:.2f}°<br>"
                        f"Zenith distance (z): {zdist:.2f}°")

def on_vert_change(change):
    sel_az = change["new"]
    for i, az in enumerate(az_options):
        fig.data[idx["vert_start"] + i].visible = (az == sel_az)
    update_star_position(alt_slider.value, sel_az)

def on_alm_toggle(change):
    show = change["new"]
    for i in range(idx["alm_start"], idx["alm_start"] + len(alm_traces)):
        fig.data[i].visible = show

def on_alt_change(change):
    update_star_position(change["new"], vert_dropdown.value)

vert_dropdown.observe(on_vert_change, names='value')
alm_checkbox.observe(on_alm_toggle, names='value')
alt_slider.observe(on_alt_change, names='value')

update_star_position(default_alt, default_az)

controls = widgets.VBox([widgets.HBox([vert_dropdown, alm_checkbox]), alt_slider, result_box])
display(fig)
display(controls)
